<a href="https://colab.research.google.com/github/FavianSulthan/latihan1/blob/main/Latihan_1_KompStat_Kelompok_6_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latihan 1 Komputasi Statistika
Kelompok 6
1. Adawia Ananda                 - 2106724883
2. Aurelio Naufal Effendy        - 2106638526
3. Favian Sulthan Wafi           - 2106706205
4. Myra Azzahra Putri Syah Indra - 2106726844
5. Rifqi Hafizuddin              - 2106638204





# Case Study
The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.

Gunakan 20 Newsgroup data berikut (Sumber Data). Bagaimana kita tahu topik apa yang dibicarakan dalam dokumen 20 Newsgroups tersebut? Mari lakukan implementasi topic modelling pada data tersebut.

# Importing Modules

In [ ]:
# Sebelum load data, kita akan unduh beberapa resources untuk kita gunakan nantinya.
# Cell hanya akan berjalan jika menggunakan Google Collab.

import warnings; warnings.simplefilter('ignore')
import nltk

try:
    import google.colab; IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataNlpTm.py
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/ind_SA.csv
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/dataset_tweet_sentiment_opini_film.csv
    !pip install spacy unidecode textblob sastrawi pyLDAvis
    !pip install --upgrade python-crfsuite gensim
    !pip install sklearn-pycrfsuite
    !python -m spacy download xx_ent_wiki_sm
    !python -m spacy download xx_sent_ud_sm
    !python -m spacy download en_core_web_sm

    nltk.download('popular')
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

--2022-09-29 08:20:48--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataNlpTm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14001 (14K) [text/plain]
Saving to: ‘taudataNlpTm.py’

taudataNlpTm.py     100%[===================>]  13.67K  --.-KB/s    in 0s      

2022-09-29 08:20:48 (88.2 MB/s) - ‘taudataNlpTm.py’ saved [14001/14001]

--2022-09-29 08:20:48--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29041 (28K) 

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
# Importing Modules untuk Notebook ini
import taudataNlpTm as tau, itertools, re, pickle, pyLDAvis, pyLDAvis.sklearn, spacy, urllib.request
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns 
from tqdm import tqdm
from nltk.tag import CRFTagger
from gensim.models import Phrases
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups

pyLDAvis.enable_notebook()
sns.set(style="ticks", color_codes=True)
random_state = 99
'Done'

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


'Done'

# Load Data

In [ ]:
# Load data 20 Newsgroup
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
D = [doc for doc in documents]
categories = dataset.target_names

# Akan kita lihat berapa banyak dokumen yang ada di data ini
print("Finished, {} documents loaded".format(len(D)))

In [ ]:
# Akan kita lihat kategori pada data
num = []
for i in range(1,len(categories)+1):
  num.append(i)
categories_df = pd.DataFrame(categories, index = num, columns =['Categories'])
print("Terdapat {} kategori pada data\n".format(len(categories)))
print(categories_df)


# Data Preprocessing

In [ ]:
# Untuk preprocessing, data akan diubah dari list ke bentuk pandas dataframe terlebih dahulu
news_df = pd.DataFrame({'document':documents})

# Mengganti tanda baca, angka, karakter spesial, dsb. dengan karakter kosong sehingga menyisakan huruf alfabet saja
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# Menghilangkan kata-kata yang pendek(kurang dari 4 huruf) karena biasanya tidak mengandung informasi yang berguna
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Mengubah semua teks menjadi huruf kecil untuk menghilangkan case sensitivity
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
# Load stopwords (kami menggunakan Spacy)
from spacy.lang.en.stop_words import STOP_WORDS as stop_words

# Tokenisasi - mengubah kalimat menjadi kata-kata sehingga dapat diproses untuk menghilangkan stopwords
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# Menghilangkan stopwords
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# Detokenisasi - menggabungkan kembali kata-kata menjadi kalimat
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc
D = detokenized_doc

# Adding LDA Matrix Decomposition

In [ ]:
# Kita mulai dengan membuat VSM-nya (Vector Space Model)
# Untuk LDA akan digunakan CountVectorizer dari sklearn untuk membuat Term-Document Matrix
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()

data = D.copy()
tf = tf_vectorizer.fit_transform(data)
tf_terms = tf_vectorizer.get_feature_names()

# Akan dilihat berapa banyak data pada dokumen
tf.shape

In [ ]:
# Selanjutnya akan dibentuk model LDA-nya
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 20 # Jumlah topik yang ingin ditelusuri. Kita ambil 20 topik karena terdapat 20 kategori pada dataset
lda = LDA(n_components=n_topics, learning_method='batch', random_state=0).fit(tf)   
vsm_topics = lda.transform(tf)
vsm_topics.shape # Terlihat bahwa 64834 data pada dokumen terkompres menjadi topiknya sehingga hanya tersisa 20 data.
                 # Hasil kompresi ini sangat baik karena sudah memuat struktur informasi pengelompokan kata pada data.
                 # Proses ini juga memudahkan dan mempercepat komputasi

# Topic Modelling

In [ ]:
# Mari kita coba maknai masing-masing topik ini
Top_Words = 25 # Jumlah kata-kata yang paling sering muncul pada masing-masing topik
print('Printing top {0} Topics, with top {1} Words:'.format(n_topics, Top_Words))
tau.print_Topics(lda, tf_terms, n_topics, Top_Words)

In [ ]:
# Selanjutnya akan kita plot agar lebih jelas
# Hasil plot dapat dilihat melalui Google Collab karena plot tidak muncul di Github
import pyLDAvis==2.1.2, pyLDAvis.sklearn; pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

Dari visualisasi tersebut, dapat disimpulkan bahwa:
*   Kata-kata yang paling menonjol dalam topik 1 antara lain: people, government, 
president, years, gun, money, dan killed. Dari kata-kata tersebut, dapat diprediksi bahwa topik 1 membicarakan tentang kebijakan pemerintahan.


*   Kata-kata yang paling menonjol dalam topik 2 antara lain: system, chip, data, encryption, information, computer, privacy, internet, public, dan security. Dari kata-kata tersebut, dapat diprediksi bahwa topik 2 membicarakan tentang keamanan cyber.


*   Kata-kata yang paling menonjol dalam topik 3 antara lain: people, jesus, believe, bible, christ, christian, faith, religion, dan hell. Dari kata-kata tersebut, dapat diprediksi bahwa topik 3 membicarakan tentang agama Kristen.

*   Kata-kata yang paling menonjol dalam topik 4 antara lain: file, windows, program, server, widget, application, system, dan information. Dari kata-kata tersebut, dapat diprediksi bahwa topik 4 membicarakan tentang sistem komputer.

*   Kata-kata yang paling menonjol dalam topik 5 antara lain: israel, government, jews, arab, public, political, dan security. Dari kata-kata tersebut, dapat diprediksi bahwa topik 5 membicarakan tentang kondisi politik Israel.

*   Kata-kata yang paling menonjol dalam topik 6 antara lain: card, windows, problem, system, software, memory, monitor, dan graphics. Dari kata-kata tersebut, dapat diprediksi bahwa topik 6 membicarakan tentang kartu grafik komputer.

*   Kata-kata yang paling menonjol dalam topik 7 antara lain:good, like, year, bike, player, average, game, dan team. Dari kata-kata tersebut, dapat diprediksi bahwa topik 7 membicarakan tentang permainan olahraga.

*   Kata-kata yang paling menonjol dalam topik 8 antara lain: people, church, science, christian, believe, catholic, dan mary. Dari kata-kata tersebut, dapat diprediksi bahwa topik 8 membicarakan tentang agama Katolik.


*   Kata-kata yang paling menonjol dalam topik 9 antara lain: team, game, hockey, season, players, league,dan rules. Dari kata-kata tersebut, dapat diprediksi bahwa topik 9 membicarakan tentang permainan hockey.


*   Kata-kata yang paling menonjol dalam topik 10 antara lain: armenian, turkish, genocide, jews, greek, muslim, islam, history, russian, soviet, ottoman, nazi, dan government. Dari kata-kata tersebut, dapat diprediksi bahwa topik 10 membicarakan tentang sejarah politik di Timur Tengah.


*   Kata-kata yang paling menonjol dalam topik 11 antara lain: time, engine, cars, miles, like, speed, dan scsi. Dari kata-kata tersebut, dapat diprediksi bahwa topik 11 membicarakan tentang mesin otomotif.


*   Kata-kata yang paling menonjol dalam topik 12 antara lain: space, nasa, launch, data, earth, satellite, shuttle, dan orbit engine. Dari kata-kata tersebut, dapat diprediksi bahwa topik 12 membicarakan tentang peluncuran ke luar angkasa.

*   Kata-kata yang paling menonjol dalam topik 13 antara lain: send, health, graphics, number, rate, mail, list, request, available, rates dan data. Dari kata-kata tersebut, dapat diprediksi bahwa topik 13 membicarakan tentang medis.

*   Kata-kata yang paling menonjol dalam topik 14 antara lain: university, april, good, center, york, national, washington, lost, internet, cancer, east, dan research. Dari kata-kata tersebut, dapat diprediksi bahwa topik 14 membicarakan tentang terdapat tempat penelitian kanker di Washington.

*   Kata-kata yang paling menonjol dalam topik 15 antara lain: patients, disease, nrhj, runs, game, wwiz, evidence, dan gizw. Dari kata-kata tersebut, dapat diprediksi bahwa topik 15 membicarakan tentang game medis.

*   Kata-kata yang paling menonjol dalam topik 16 antara lain: water, food, candida, cause, blood, father, doctor, body, dan medical. Dari kata-kata tersebut, dapat diprediksi bahwa topik 16 membicarakan tentang donasi untuk keperluan medis.

*   Kata-kata yang paling menonjol dalam topik 17 antara lain: period, play, power, flyers, second, scorer, shots, dan goal. Dari kata-kata tersebut, dapat diprediksi bahwa topik 17 membicarakan tentang liga hoki nasional Amerika Serikat.


*   Kata-kata yang paling menonjol dalam topik 18 antara lain: drive, disk, drives, hard, scsi, controller, floppy, bios, slave, dan system. Dari kata-kata tersebut, dapat diprediksi bahwa topik 18 membicarakan tentang elektronik.


*   Kata-kata yang paling menonjol dalam topik 19 antara lain: know, problem, work, xfree, like, problems, dan senior. Dari kata-kata tersebut, dapat diprediksi bahwa topik 19 membicarakan tentang permasalahan di kantor.


*   Kata-kata yang paling menonjol dalam topik 20 antara lain: like, echo, think, mahan, daughter, maria, corn, bayonet, dan bullets. Dari kata-kata tersebut, dapat diprediksi bahwa topik 20 membicarakan tentang Mahan, seorang perwira angkatan laut amerika serikat.





























Referensi: 
*   https://github.com/gnespatel1618/TopicModeling/blob/master/Topic%20Modeling.ipynb
*   https://taudata.blogspot.com/2022/05/nlptm-07.html

